In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as kb
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Activation, Add, Lambda
from tensorflow.keras.layers import Dense, MaxPooling1D, Conv1D, LSTM
from tensorflow.keras.backend import ctc_batch_cost
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
train_X = np.load("train_X.npy")
train_y = np.load("train_y.npy", allow_pickle=True)
# test_X  = np.load("test_X.npy")
# test_y  = np.load("test_y.npy", allow_pickle=True)

print(f"input shape: {train_X.shape}")
print(train_y.shape)
# print(test_X.shape)
# print(test_y.shape)

train_X_lens = np.array([[97] for x in train_X], dtype="float32")
print(f"input_length shape: {train_X_lens.shape}")

train_y_lens = np.array([[len(x)] for x in train_y], dtype="float32")
print(f"label_length shape: {train_y_lens.shape}")

maxlen = max([len(r) for r in train_y])
train_y_padded = np.array([r + [-1]*(maxlen-len(r)) for r in train_y], dtype='float32')
print(f"labels shape: {train_y_padded.shape}")

In [ ]:
inputs = {'the_input': train_X,
          'the_labels': train_y_padded,
          'input_length': train_X_lens,
          'label_length': train_y_lens
          }
outputs = {'ctc': np.zeros([len(train_X)])}  # dummy data for dummy loss function
training_data = (inputs, outputs)

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return kb.ctc_batch_cost(
        y_true=labels, 
        y_pred=y_pred, 
        input_length=input_length, 
        label_length=label_length
    )

In [ ]:
input_data = Input(name="the_input", shape=(200,1), dtype="float32")
inner = Conv1D(32, 3,
          padding="valid",
          activation="relu",
          name="conv1d_1")(input_data)
inner = MaxPooling1D(pool_size=2, name="maxpool_1")(inner)
lstm_1a = LSTM(100,return_sequences=True, name="lstm_1a")(inner)
lstm_1b = LSTM(100, return_sequences=True, go_backwards=True, name="lstm_1b")(inner)
lstm_1_merged = Add()([lstm_1a, lstm_1b])

inner = Dense(5, name="dense_1")(lstm_1_merged)

y_pred = Activation("softmax", name="softmax")(inner)

# Model(inputs=input_data, outputs=y_pred).summary()

labels = Input(name='the_labels', shape=(maxlen), dtype='float32')
input_length = Input(name='input_length', shape=(1), dtype='int64')
label_length = Input(name='label_length', shape=(1), dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out, name="my_model")
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

# model.summary()


In [ ]:
model.fit(x=inputs, y=outputs)

In [ ]:
inputs['the_input'][0]

In [ ]:
pred = model.predict(inputs['the_input'][0])

In [ ]:
batchsize = 100

def testgen(train_X, train_y, train_X_lens, train_y_lens, batchsize):
    i = 0
    while i + batchsize < len(train_X):
        inputs = {'the_input': train_X[i:i+batchsize],
          'the_labels': train_y_padded[i:i+batchsize],
          'input_length': train_X_lens[i:i+batchsize],
          'label_length': train_y_lens[i:i+batchsize]
          }
        outputs = {'ctc': np.zeros([batchsize])}
        i+=batchsize
        yield (inputs, outputs)

model.fit_generator(generator=testgen(train_X, train_y, train_X_lens, train_y_lens, batchsize), steps_per_epoch=(len(train_X)//batchsize))

In [ ]:
g = testgen(train_X, train_y, train_X_lens, train_y_lens)

In [ ]:
generated = next(g)

In [ ]:
print(generated[0]['the_labels'][0]) # list of length $max_string_length with the string
print(generated[0]['label_length'][0]) # single element noting the length of the label
im0 = generated[0]['the_input'][0]
print(im0.shape)
print(im0)